### preprocess civic

In [1]:
import pandas as pd
import numpy as np

In [2]:
civic_df = pd.read_csv("../datasets/rawData-21-Dec-2022/civic/nightly-ClinicalEvidenceSummaries.tsv", sep='\t', on_bad_lines='skip')
print(civic_df)
civic_df.head()

        gene  entrez_id variant                         disease    doid  \
0       JAK2       3717   V617F               Lymphoid Leukemia  1037.0   
1     PDGFRA       5156   D842V  Gastrointestinal Stromal Tumor  9253.0   
2     DNMT3A       1788    R882          Acute Myeloid Leukemia  9119.0   
3     DNMT3A       1788    R882          Acute Myeloid Leukemia  9119.0   
4       JAK2       3717   V617F        Chronic Myeloid Leukemia  8552.0   
...      ...        ...     ...                             ...     ...   
4030    TP53       7157   G266V                             NaN     NaN   
4031    TP53       7157   G266E                             NaN     NaN   
4032    TP53       7157   F270S                             NaN     NaN   
4033    TP53       7157   V272G                             NaN     NaN   
4034    IDH1       3417    R132              Cholangiocarcinoma  4947.0   

     phenotypes       drugs drug_interaction_type evidence_type  \
0           NaN         NaN     

,gene,entrez_id,variant,disease,doid,phenotypes,drugs,drug_interaction_type,evidence_type,evidence_direction,...,representative_transcript2,ensembl_version,reference_build,variant_summary,variant_origin,last_review_date,evidence_civic_url,variant_civic_url,gene_civic_url,is_flagged
0,JAK2,3717,V617F,Lymphoid Leukemia,1037.0,NaN,NaN,NaN,Diagnostic,Supports,...,NaN,75.0,GRCh37,JAK2 V617F is a highly recurrent mutation in m...,Somatic,2020-11-20 16:18:21 UTC,https://civicdb.org/links/evidence_items/1,https://civicdb.org/links/variants/64,https://civicdb.org/links/genes/28,False
1,PDGFRA,5156,D842V,Gastrointestinal Stromal Tumor,9253.0,NaN,NaN,NaN,Diagnostic,Supports,...,NaN,75.0,GRCh37,PDGFRA D842 mutations are characterized broadl...,Somatic,2015-06-21 16:49:38 UTC,https://civicdb.org/links/evidence_items/2,https://civicdb.org/links/variants/99,https://civicdb.org/links/genes/38,False
2,DNMT3A,1788,R882,Acute Myeloid Leukemia,9119.0,NaN,NaN,NaN,Diagnostic,Supports,...,NaN,75.0,GRCh37,DNMT3A R882 mutations are associated with cyto...,Somatic,2016-03-26 22:10:59 UTC,https://civicdb.org/links/evidence_items/3,https://civicdb.org/links/variants/32,https://civicdb.org/links/genes/18,False
3,DNMT3A,1788,R882,Acute Myeloid Leukemia,9119.0,NaN,NaN,NaN,Diagnostic,Supports,...,NaN,75.0,GRCh37,DNMT3A R882 mutations are associated with cyto...,Somatic,2016-03-16 22:09:27 UTC,https://civicdb.org/links/evidence_items/4,https://civicdb.org/links/variants/32,https://civicdb.org/links/genes/18,False
4,JAK2,3717,V617F,Chronic Myeloid Leukemia,8552.0,NaN,NaN,NaN,Diagnostic,Supports,...,NaN,75.0,GRCh37,JAK2 V617F is a highly recurrent mutation in m...,Somatic,2015-06-21 16:49:39 UTC,https://civicdb.org/links/evidence_items/5,https://civicdb.org/links/variants/64,https://civicdb.org/links/genes/28,False


#### Part 1: drugs & evidence_direction filtering

In [69]:
# predictive_df = civic_df[civic_df['evidence_type'] == 'Predictive']
predictive_df = civic_df.dropna(subset=['drugs'])
print(predictive_df.shape)
support_df = predictive_df[predictive_df['evidence_direction'] == 'Supports']
print(support_df.shape)

(2485, 42)
(2121, 42)


#### Part 2: variant filtering

In [51]:
# check regular expression
test_list = ['A325B','A40','A3820BC','AODBS','AB492C']
import re
pattern = re.compile('[A-Z][0-9]+[A-Z]')
for item in test_list:
    print(pattern.findall(item))


['A325B']
[]
['A3820B']
[]
['B492C']


In [104]:
#missense_df = support_df[support_df['start'] == support_df['stop']]
#print(missense_df.shape)


missense_df = support_df.dropna(subset=['start','stop'])
columns = ['gene', 'variant', 'drugs', 'clinical_significance', 'chromosome', 'start', 'stop', 'reference_bases', 'variant_bases', 'representative_transcript', 'drug_interaction_type']
missense_df = pd.DataFrame(data=missense_df, columns=columns)
missense_df.drop_duplicates(inplace=True)

# check missense
missense_df['variant'] = missense_df['variant'].str.split(' ')
missense_df['variant'] = missense_df['variant'].apply(lambda x: x[-1])
missense_df['variant'] = missense_df['variant'].str.split('::')
missense_df['variant'] = missense_df['variant'].apply(lambda x: x[-1])

pd.options.display.max_rows = 20
# update: regular expression filtering

missense_df = missense_df[missense_df['variant'].str.contains('[A-Z][0-9]+[A-Z]')]
missense_df = missense_df[~missense_df['variant'].str.contains('_')]
missense_df = missense_df[~missense_df['variant'].str.contains('DEL')]
missense_df = missense_df[~missense_df['variant'].str.contains('INS')]
missense_df = missense_df[~missense_df['variant'].str.contains('INST')]
missense_df = missense_df[~missense_df['variant'].str.contains('-')]
missense_df = missense_df[~missense_df['variant'].str.contains('/')].reset_index(drop=True)

sele_index = [38, 176, 252, 260, 269, 286, 550]
missense_df.drop(index=sele_index, inplace=True)
print(missense_df.shape)
#print(missense_df)
missense_df

(711, 11)


,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript,drug_interaction_type
0,MAP2K1,P124S,Selumetinib,Resistance,15,66729162.0,66729162.0,C,T,ENST00000307102.5,NaN
1,MAP2K1,Q56P,Selumetinib,Resistance,15,66727451.0,66727451.0,A,C,ENST00000307102.5,NaN
2,PDGFRA,D842V,Imatinib,Resistance,4,55152093.0,55152093.0,A,T,ENST00000257290.5,NaN
3,ARAF,S214C,Sorafenib,Sensitivity/Response,X,47426121.0,47426121.0,C,G,ENST00000377045.4,NaN
4,JAK2,V617F,Peginterferon Alfa-2b,Sensitivity/Response,9,5073770.0,5073770.0,G,T,ENST00000381652.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...
713,BRAF,V600E,"Encorafenib,Cetuximab",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
714,EGFR,G598V,Erlotinib,Sensitivity/Response,7,55233043.0,55233043.0,G,T,ENST00000275493.2,NaN
715,SF3B1,K700E,"Etoposide,Olaparib",Sensitivity/Response,2,198266834.0,198266834.0,T,C,ENST00000335508.6,Substitutes
716,IDH1,R132C,Olaparib,Sensitivity/Response,2,209113113.0,209113113.0,G,A,ENST00000415913.1,NaN


#### Part 3: drug filtering

In [105]:
# view unique : original 234 drugs
missense_df['drugs'].value_counts()

Imatinib                         40
Dasatinib                        29
Imatinib Mesylate                28
Erlotinib                        24
Cetuximab                        20
                                 ..
MTOR Kinase Inhibitor AZD8055     1
PLX4720                           1
Lapatinib,Capivasertib            1
Capivasertib,Trastuzumab          1
Pemigatinib                       1
Name: drugs, Length: 234, dtype: int64

In [144]:
# search drug combination
combination_df = missense_df.dropna(subset=['drug_interaction_type'], inplace=False).reset_index(drop=True)
combination_df.to_csv('../datasets/middlefile/civic_drug_combination.csv', index=None)
combination_df

,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript,drug_interaction_type
0,KIT,L576P,"Sorafenib,Imatinib,Nilotinib",Resistance,4,55593661.0,55593661.0,T,C,ENST00000288135.5,Substitutes
1,BRAF,V600E,"Sorafenib,Panitumumab",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
2,BRAF,V600E,"Dabrafenib,Trametinib",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
3,BRAF,V600E,"Pictilisib Bismesylate,PLX4720",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
4,BRAF,V600E,"Nutlin-3,PLX4720",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
...,...,...,...,...,...,...,...,...,...,...,...
203,PIK3CA,H1047R,"Akt Inhibitor MK2206,Vemurafenib",Sensitivity/Response,3,178952085.0,178952085.0,A,G,ENST00000263967.3,Combination
204,BRAF,V600E,"Irinotecan,Vemurafenib,Cetuximab",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
205,BRAF,V600E,"Erlotinib,Vemurafenib",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination
206,BRAF,V600E,"Encorafenib,Cetuximab",Sensitivity/Response,7,140453136.0,140453136.0,A,T,ENST00000288602.6,Combination


In [107]:
singledrug_df = missense_df[missense_df['drug_interaction_type'].isnull()]
columns = ['gene', 'variant', 'drugs', 'clinical_significance', 'chromosome', 'start', 'stop', 'reference_bases', 'variant_bases', 'representative_transcript']
newsingledrug_df = pd.DataFrame(data=singledrug_df, columns=columns)
newsingledrug_df.drop_duplicates(inplace=True)
newsingledrug_df = newsingledrug_df.reset_index(drop=True)
newsingledrug_df

,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript
0,MAP2K1,P124S,Selumetinib,Resistance,15,66729162.0,66729162.0,C,T,ENST00000307102.5
1,MAP2K1,Q56P,Selumetinib,Resistance,15,66727451.0,66727451.0,A,C,ENST00000307102.5
2,PDGFRA,D842V,Imatinib,Resistance,4,55152093.0,55152093.0,A,T,ENST00000257290.5
3,ARAF,S214C,Sorafenib,Sensitivity/Response,X,47426121.0,47426121.0,C,G,ENST00000377045.4
4,JAK2,V617F,Peginterferon Alfa-2b,Sensitivity/Response,9,5073770.0,5073770.0,G,T,ENST00000381652.3
...,...,...,...,...,...,...,...,...,...,...
496,CSF1R,CSF1R,Imatinib,Sensitivity/Response,1,156446804.0,156470620.0,NaN,NaN,ENST00000348159.4
497,EZH2,Y646H,Tazemetostat,Sensitivity/Response,7,148508728.0,148508728.0,A,G,ENST00000320356.7
498,EGFR,G598V,Erlotinib,Sensitivity/Response,7,55233043.0,55233043.0,G,T,ENST00000275493.2
499,IDH1,R132C,Olaparib,Sensitivity/Response,2,209113113.0,209113113.0,G,A,ENST00000415913.1


In [108]:
# check significance
newsingledrug_df['clinical_significance'].value_counts()

Resistance              250
Sensitivity/Response    239
Reduced Sensitivity      10
Adverse Response          2
Name: clinical_significance, dtype: int64

#### Part 4: pubchem drug mapping

In [110]:
drug_unique_list = newsingledrug_df['drugs'].unique()
print(len(drug_unique_list))
drug_unique_list

108


array(['Selumetinib', 'Imatinib', 'Sorafenib', 'Peginterferon Alfa-2b',
       'Fedratinib', 'Tanespimycin', 'Temozolomide', 'Crizotinib',
       'Alectinib', 'TAE684', 'Trametinib', 'Dasatinib', 'Crenolanib',
       'Dabrafenib', 'Motesanib', 'JAK2 Inhibitor AZD1480', 'Vemurafenib',
       'Cetuximab', 'NSC348884', 'Erlotinib', 'Gefitinib', 'Dacomitinib',
       'Lapatinib', 'Hormone Therapy', 'Staurosporine', 'Neratinib',
       'Sunitinib', 'PI3K-alpha Inhibitor MEN1611', 'Sirolimus',
       'Doxorubicin', 'Ibrutinib', 'Ponatinib', 'Rociletinib', 'Olaparib',
       'Fluorouracil', 'FOLFOX Regimen', 'Paclitaxel', 'Cisplatin',
       'Epidermal Growth Factor Receptor Tyrosine Kinase Inhibitor',
       'Uprosertib', 'Capivasertib', 'GSK321', 'Idelalisib', 'Vismodegib',
       'Pictilisib', 'Abemaciclib', 'Palbociclib', 'Lorlatinib',
       'Afatinib', 'Akt Inhibitor MK2206', 'Amrubicin', 'Osimertinib',
       'AGI-5198', 'Futuximab/Modotuximab Mixture', 'Panitumumab',
       'Tretinoin

In [118]:
import pubchempy as pcp
from tqdm import tqdm

# test
#results = pcp.get_compounds('Selumetinib','name') # [Compound(10127622)]

error_list = []
for i in tqdm(range(len(drug_unique_list))):
    if(len(pcp.get_compounds(drug_unique_list[i],'name')) == 0):
        error_list.append(drug_unique_list[i])
error_list

100%|██████████| 108/108 [02:20<00:00,  1.31s/it]


['Peginterferon Alfa-2b',
 'JAK2 Inhibitor AZD1480',
 'Cetuximab',
 'Hormone Therapy',
 'PI3K-alpha Inhibitor MEN1611',
 'Epidermal Growth Factor Receptor Tyrosine Kinase Inhibitor',
 'Akt Inhibitor MK2206',
 'Futuximab/Modotuximab Mixture',
 'Panitumumab',
 'Trastuzumab',
 'MTOR Kinase Inhibitor AZD8055',
 'Aromatase Inhibitor',
 'Therapeutic Tumor Infiltrating Lymphocytes',
 'EAP Protocol',
 'MDM2 Inhibitor AMGMDS3',
 'Akt/ERK Inhibitor ONC201',
 'PI3Ka/Di']

In [134]:
# fix drugname for pubchem
fixdrug_df = newsingledrug_df[~newsingledrug_df['drugs'].str.contains('Peginterferon Alfa-2b')]
#fix_druglist = ['AZD1480' if i == 'JAK2 Inhibitor AZD1480' else i for i in drug_unique_list]
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('JAK2 Inhibitor AZD1480', 'AZD1480')
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('Cetuximab', 'Erbitux')
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Hormone Therapy')]
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('PI3K-alpha Inhibitor MEN1611', 'MEN1611')
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Epidermal Growth Factor Receptor Tyrosine Kinase Inhibitor')]
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('Akt Inhibitor MK2206', 'MK2206')
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('Trastuzumab', 'BCP32858')
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('MTOR Kinase Inhibitor AZD8055', 'AZD8055')
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Aromatase Inhibitor')]
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Therapeutic Tumor Infiltrating Lymphocytes')]
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('EAP Protocol')]
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('MDM2 Inhibitor AMGMDS3', 'AMGMDS3')
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('Akt/ERK Inhibitor ONC201', 'ONC201')
fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('Akt/ERK Inhibitor ONC201', 'ONC201')
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('PI3Ka/Di')]
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Futuximab/Modotuximab Mixture')]
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Panitumumab')]
fixdrug_df = fixdrug_df[~fixdrug_df['drugs'].str.contains('Erbitux')]


new_drug_unique_list = fixdrug_df['drugs'].unique()
new_error_list = []
for i in tqdm(range(len(new_drug_unique_list))):
    if(len(pcp.get_compounds(new_drug_unique_list[i],'name')) == 0):
        new_error_list.append(new_drug_unique_list[i])
new_error_list

C:\Users\Administrator\AppData\Local\Temp\ipykernel_3240\1316244949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('JAK2 Inhibitor AZD1480', 'AZD1480')
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3240\1316244949.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixdrug_df['drugs'] = fixdrug_df['drugs'].str.replace('Cetuximab', 'Erbitux')
100%|██████████| 98/98 [02:07<00:00,  1.31s/it]


[]

In [145]:
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 20

# generate drug table: smile
drug_table = pd.DataFrame(columns=['drugname', 'smile', 'molecular_weight', 'molecular_formula', 'atom'])
for i in tqdm(range(len(new_drug_unique_list))):
    compound = pcp.get_compounds(new_drug_unique_list[i],'name')[0]
    drugname = new_drug_unique_list[i]
    try:
        smile = compound.isomeric_smiles
    except AttributeError:
        smile = np.nan
    try:
        molecular_weight = compound.molecular_weight
    except AttributeError:
        molecular_weight = np.nan   
    try:
        molecular_formula = compound.molecular_formula
    except AttributeError:
        molecular_formula = np.nan
    try: 
        atom = compound.atoms
    except AttributeError:
        atom = np.nan    
    drug_table = drug_table.append([{'drugname':drugname, 'smile':smile, 'molecular_weight':molecular_weight, 'molecular_formula':molecular_formula, 
                                    'atom':atom}], ignore_index=True)
print(drug_table)
drug_table.to_csv('../datasets/middlefile/civic_drug_table.csv', index=None)

100%|██████████| 98/98 [02:06<00:00,  1.29s/it]

            drugname                                              smile  \
0        Selumetinib  CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...   
1           Imatinib  CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...   
2          Sorafenib  CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...   
3         Fedratinib  CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...   
4       Tanespimycin  C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H]...   
..               ...                                                ...   
93  Arsenic Trioxide                    [O-2].[O-2].[O-2].[As+3].[As+3]   
94        Venetoclax  CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC...   
95      Gilteritinib  CCC1=C(N=C(C(=N1)C(=O)N)NC2=CC(=C(C=C2)N3CCC(C...   
96      Tazemetostat  CCN(C1CCOCC1)C2=CC(=CC(=C2C)C(=O)NCC3=C(C=C(NC...   
97       Pemigatinib  CCN1C2=C3C=C(NC3=NC=C2CN(C1=O)C4=C(C(=CC(=C4F)...   

   molecular_weight molecular_formula  \
0             457.7   C17H15BrClFN4O3   
1             493

In [147]:
drug_table

,drugname,smile,molecular_weight,molecular_formula,atom
0,Selumetinib,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,457.7,C17H15BrClFN4O3,"[Atom(1, Br), Atom(2, Cl), Atom(3, F), Atom(4,..."
1,Imatinib,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,493.6,C29H31N7O,"[Atom(1, O), Atom(2, N), Atom(3, N), Atom(4, N..."
2,Sorafenib,CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...,464.8,C21H16ClF3N4O3,"[Atom(1, Cl), Atom(2, F), Atom(3, F), Atom(4, ..."
3,Fedratinib,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,524.7,C27H36N6O3S,"[Atom(1, S), Atom(2, O), Atom(3, O), Atom(4, O..."
4,Tanespimycin,C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H]...,585.7,C31H43N3O8,"[Atom(1, O), Atom(2, O), Atom(3, O), Atom(4, O..."
...,...,...,...,...,...
93,Arsenic Trioxide,[O-2].[O-2].[O-2].[As+3].[As+3],197.841,As2O3,"[Atom(1, As), Atom(2, As), Atom(3, O), Atom(4,..."
94,Venetoclax,CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC...,868.4,C45H50ClN7O7S,"[Atom(1, Cl), Atom(2, S), Atom(3, O), Atom(4, ..."
95,Gilteritinib,CCC1=C(N=C(C(=N1)C(=O)N)NC2=CC(=C(C=C2)N3CCC(C...,552.7,C29H44N8O3,"[Atom(1, O), Atom(2, O), Atom(3, O), Atom(4, N..."
96,Tazemetostat,CCN(C1CCOCC1)C2=CC(=CC(=C2C)C(=O)NCC3=C(C=C(NC...,572.7,C34H44N4O4,"[Atom(1, O), Atom(2, O), Atom(3, O), Atom(4, O..."


In [146]:
fixdrug_df.to_csv('../datasets/middlefile/civic_evidence_table.csv', index=None)
fixdrug_df

,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript
0,MAP2K1,P124S,Selumetinib,Resistance,15,66729162.0,66729162.0,C,T,ENST00000307102.5
1,MAP2K1,Q56P,Selumetinib,Resistance,15,66727451.0,66727451.0,A,C,ENST00000307102.5
2,PDGFRA,D842V,Imatinib,Resistance,4,55152093.0,55152093.0,A,T,ENST00000257290.5
3,ARAF,S214C,Sorafenib,Sensitivity/Response,X,47426121.0,47426121.0,C,G,ENST00000377045.4
5,JAK2,V617F,Fedratinib,Sensitivity/Response,9,5073770.0,5073770.0,G,T,ENST00000381652.3
...,...,...,...,...,...,...,...,...,...,...
496,CSF1R,CSF1R,Imatinib,Sensitivity/Response,1,156446804.0,156470620.0,NaN,NaN,ENST00000348159.4
497,EZH2,Y646H,Tazemetostat,Sensitivity/Response,7,148508728.0,148508728.0,A,G,ENST00000320356.7
498,EGFR,G598V,Erlotinib,Sensitivity/Response,7,55233043.0,55233043.0,G,T,ENST00000275493.2
499,IDH1,R132C,Olaparib,Sensitivity/Response,2,209113113.0,209113113.0,G,A,ENST00000415913.1


In [149]:
fixdrug_df['clinical_significance'].value_counts()

Sensitivity/Response    225
Resistance              218
Reduced Sensitivity      10
Adverse Response          2
Name: clinical_significance, dtype: int64

#### Part 5: gene variant mapping

In [150]:
fixdrug_df['gene'].value_counts()

ABL1      108
EGFR       50
ALK        36
PIK3CA     30
KIT        26
         ... 
GSTP1       1
MTHFR       1
NPM1        1
JAK2        1
EZH2        1
Name: gene, Length: 57, dtype: int64

In [151]:
gene_list = fixdrug_df['gene'].unique()
gene_list

array(['MAP2K1', 'PDGFRA', 'ARAF', 'JAK2', 'NRAS', 'ALK', 'KIT', 'BRAF',
       'RET', 'NPM1', 'ABL1', 'EGFR', 'ERBB2', 'DDR2', 'PIK3CA', 'TP53',
       'BTK', 'AKT1', 'ATM', 'MTHFR', 'GSTP1', 'ABCB1', 'ERCC2', 'IDH1',
       'SMO', 'MAPK1', 'KRAS', 'NQO1', 'MYD88', 'PML', 'ROS1', 'MTOR',
       'KDR', 'FGFR1', 'FLT3', 'FGFR2', 'CDK4', 'SF3B1', 'GNAQ', 'FGFR3',
       'STK11', 'MET', 'ERBB3', 'SLCO1B1', 'NTRK3', 'JAK1', 'BRCA2',
       'NTRK1', 'HRAS', 'AKT3', 'BRCA1', 'PREX2', 'CSF3R', 'H3-3A',
       'BCL2', 'CSF1R', 'EZH2'], dtype=object)

In [166]:
import requests
import re

# test
url = 'https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=(reviewed:true)%20AND%20(organism_id:9606)%20AND%20MAP2K1'
all_fastas = requests.get(url).text
fasta_list = re.split(r'\n(?=>)', all_fastas)
entry = [fasta for fasta in fasta_list if 'MAP2K1' in fasta][0]
uniprotac = entry.split('|')[1]
fasta = entry.split(' ')[-1].replace('\n','')[4:]
print(uniprotac, fasta)

Q02750 MPKKKPTPIQLNPAPDGSAVNGTSSAETNLEALQKKLEELELDEQQRKRLEAFLTQKQKVGELKDDDFEKISELGAGNGGVVFKVSHKPSGLVMARKLIHLEIKPAIRNQIIRELQVLHECNSPYIVGFYGAFYSDGEISICMEHMDGGSLDQVLKKAGRIPEQILGKVSIAVIKGLTYLREKHKIMHRDVKPSNILVNSRGEIKLCDFGVSGQLIDSMANSFVGTRSYMSPERLQGTHYSVQSDIWSMGLSLVEMAVGRYPIPPPDAKELELMFGCQVEGDAAETPPRPRTPGRPLSSYGMDSRPPMAIFELLDYIVNEPPPKLPSGVFSLEFQDFVNKCLIKNPAERADLKQLMVHAFIKRSDAEEVDFAGWLCSTIGLNQPSTPTHAAGV


In [174]:
# fix error H3-3A: for drug table and evidence table
error_gene = 'H3-3A'
error_drug = 'ONC201'
fixdrug_df = fixdrug_df[~fixdrug_df['gene'].str.contains(error_gene)]
fixdrug_df = fixdrug_df.dropna(subset=['reference_bases','variant_bases']).reset_index(drop=True)
fixdrug_df.to_csv('../datasets/middlefile/civic_evidence_table.csv', index=None)
fixdrug_df

,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript
0,MAP2K1,P124S,Selumetinib,Resistance,15,66729162.0,66729162.0,C,T,ENST00000307102.5
1,MAP2K1,Q56P,Selumetinib,Resistance,15,66727451.0,66727451.0,A,C,ENST00000307102.5
2,PDGFRA,D842V,Imatinib,Resistance,4,55152093.0,55152093.0,A,T,ENST00000257290.5
3,ARAF,S214C,Sorafenib,Sensitivity/Response,X,47426121.0,47426121.0,C,G,ENST00000377045.4
4,JAK2,V617F,Fedratinib,Sensitivity/Response,9,5073770.0,5073770.0,G,T,ENST00000381652.3
...,...,...,...,...,...,...,...,...,...,...
445,ABL1,H396P,Dasatinib,Sensitivity/Response,9,133750356.0,133750356.0,A,C,ENST00000318560.5
446,EZH2,Y646H,Tazemetostat,Sensitivity/Response,7,148508728.0,148508728.0,A,G,ENST00000320356.7
447,EGFR,G598V,Erlotinib,Sensitivity/Response,7,55233043.0,55233043.0,G,T,ENST00000275493.2
448,IDH1,R132C,Olaparib,Sensitivity/Response,2,209113113.0,209113113.0,G,A,ENST00000415913.1


In [169]:
drug_table = drug_table[~drug_table['drugname'].str.contains(error_drug)]
drug_table.to_csv('../datasets/middlefile/civic_drug_table.csv', index=None)
drug_table

,drugname,smile,molecular_weight,molecular_formula,atom
0,Selumetinib,CN1C=NC2=C1C=C(C(=C2F)NC3=C(C=C(C=C3)Br)Cl)C(=...,457.7,C17H15BrClFN4O3,"[Atom(1, Br), Atom(2, Cl), Atom(3, F), Atom(4,..."
1,Imatinib,CC1=C(C=C(C=C1)NC(=O)C2=CC=C(C=C2)CN3CCN(CC3)C...,493.6,C29H31N7O,"[Atom(1, O), Atom(2, N), Atom(3, N), Atom(4, N..."
2,Sorafenib,CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...,464.8,C21H16ClF3N4O3,"[Atom(1, Cl), Atom(2, F), Atom(3, F), Atom(4, ..."
3,Fedratinib,CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)...,524.7,C27H36N6O3S,"[Atom(1, S), Atom(2, O), Atom(3, O), Atom(4, O..."
4,Tanespimycin,C[C@H]1C[C@@H]([C@@H]([C@H](/C=C(/[C@@H]([C@H]...,585.7,C31H43N3O8,"[Atom(1, O), Atom(2, O), Atom(3, O), Atom(4, O..."
...,...,...,...,...,...
93,Arsenic Trioxide,[O-2].[O-2].[O-2].[As+3].[As+3],197.841,As2O3,"[Atom(1, As), Atom(2, As), Atom(3, O), Atom(4,..."
94,Venetoclax,CC1(CCC(=C(C1)C2=CC=C(C=C2)Cl)CN3CCN(CC3)C4=CC...,868.4,C45H50ClN7O7S,"[Atom(1, Cl), Atom(2, S), Atom(3, O), Atom(4, ..."
95,Gilteritinib,CCC1=C(N=C(C(=N1)C(=O)N)NC2=CC(=C(C=C2)N3CCC(C...,552.7,C29H44N8O3,"[Atom(1, O), Atom(2, O), Atom(3, O), Atom(4, N..."
96,Tazemetostat,CCN(C1CCOCC1)C2=CC(=CC(=C2C)C(=O)NCC3=C(C=C(NC...,572.7,C34H44N4O4,"[Atom(1, O), Atom(2, O), Atom(3, O), Atom(4, O..."


In [199]:
gene_list = fixdrug_df['gene'].unique()
gene_table = pd.DataFrame(columns=['gene', 'uniprotac', 'fasta'])
for i in tqdm(range(len(gene_list))):
    url = 'https://rest.uniprot.org/uniprotkb/stream?compressed=false&format=fasta&query=(reviewed:true)%20AND%20(organism_id:9606)%20AND%20' + gene_list[i]
    all_fastas = requests.get(url).text
    fasta_list = re.split(r'\n(?=>)', all_fastas)
    entry = [fasta for fasta in fasta_list if gene_list[i] in fasta][0] # fix: use 'GN='+gene_list[i] instead of gene_list[i]
    gene = gene_list[i]
    uniprotac = entry.split('|')[1]
    fasta = entry.split(' ')[-1].replace('\n','')[4:]
    gene_table = gene_table.append([{'gene':gene, 'uniprotac':uniprotac, 'fasta':fasta}], ignore_index=True)
print(gene_table)
gene_table.to_csv('../datasets/middlefile/civic_gene_table.csv', index=None)

100%|██████████| 53/53 [01:55<00:00,  2.18s/it]

      gene uniprotac                                              fasta
0   MAP2K1    Q02750  MPKKKPTPIQLNPAPDGSAVNGTSSAETNLEALQKKLEELELDEQQ...
1   PDGFRA    P16234  MGTSHPAFLVLGCLLTGLSLILCQLSLPSILPNENEKVVQLNSSFS...
2     ARAF    P10398  MEPPRGPPANGAEPSRAVGTVKVYLPNKQRTVVTVRDGMSVYDSLD...
3     JAK2    O60674  MGMACLTMTEMEGTSTSSIYQNGDISGNANSMKQIDPVLQVYLYHS...
4     NRAS    P01111  MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEYDPTIEDSYRKQVVI...
..     ...       ...                                                ...
48   BRCA1    P38398  MDLSALRVEEVQNVINAMQKILECPICLELIKEPVSTKCDHIFCKF...
49   PREX2    Q70Z35  MSEDSRGDSRAESAKDLEKQLRLRVCVLSELQKTERDYVGTLEFLV...
50   CSF3R    Q99062  MARLGNCSLTWAALIILLLPGSLEECGHISVSAPIVHLGDPITASC...
51    BCL2    O43521  MAKQPSDVSSECDREGRQLQPAERPPQLRPGAPTSLQTEPQGNPEG...
52    EZH2    Q15910  MGQTGKKSEKGPVCWRKRVKSEYMRLRQLKRFRRADEVKSMFSSNR...

[53 rows x 3 columns]


In [176]:
# check variant with fasta
check_variant_df = fixdrug_df[['gene','variant']]
check_variant_df

,gene,variant
0,MAP2K1,P124S
1,MAP2K1,Q56P
2,PDGFRA,D842V
3,ARAF,S214C
4,JAK2,V617F
...,...,...
445,ABL1,H396P
446,EZH2,Y646H
447,EGFR,G598V
448,IDH1,R132C


In [178]:
protein_dict = {'C':0, 'D':1, 'S':2, 'Q':3, 'K':4,
        'I':5, 'P':6, 'T':7, 'F':8, 'N':9,
        'G':10, 'H':11, 'L':12, 'R':13, 'W':14,
        'A':15, 'V':16, 'E':17, 'Y':18, 'M':19}

In [200]:
gene_table_manu = pd.read_csv('../datasets/middlefile/civic_gene_table_fixed.csv')
for i in range(len(check_variant_df)):
    gene = check_variant_df['gene'][i]
    #print(gene)
    variant = check_variant_df['variant'][i]
    fasta = gene_table_manu[gene_table_manu['gene'] == gene]['fasta'].values[0]
    #print(fasta)
    pos_before = variant[0]
    pos = int(variant[1:-1])
    #print(pos)
    pos_after = variant[-1]
    
    # check pos_after
    if(pos_after not in protein_dict.keys()):
        print('pos_after', gene, variant)
    # check pos_before
    if(fasta[pos-1] != pos_before):
        print('pos_before', gene, variant)
        

pos_before MYD88 L265P
pos_before FGFR2 N550K
pos_after BRCA1 W1815X
pos_before ABL1 R351W
pos_before ABL1 R351W
pos_before EZH2 Y646H


In [202]:
# remove error evidence
fix_evidence_df = fixdrug_df.copy()
for i in range(len(check_variant_df)):
    gene = check_variant_df['gene'][i]
    variant = check_variant_df['variant'][i]
    fasta = gene_table_manu[gene_table_manu['gene'] == gene]['fasta'].values[0]
    pos_before = variant[0]
    pos = int(variant[1:-1])
    pos_after = variant[-1]
    
    # check pos_after
    if(pos_after not in protein_dict.keys()):
        print('pos_after', gene, variant)
        fix_evidence_df = fix_evidence_df[~((fix_evidence_df['gene'] == gene) & (fix_evidence_df['variant'] == variant))]
    # check pos_before
    if(fasta[pos-1] != pos_before):
        print('pos_before', gene, variant)
        fix_evidence_df = fix_evidence_df[~((fix_evidence_df['gene'] == gene) & (fix_evidence_df['variant'] == variant))]
fix_evidence_df

pos_before MYD88 L265P
pos_before FGFR2 N550K
pos_after BRCA1 W1815X
pos_before ABL1 R351W
pos_before ABL1 R351W
pos_before EZH2 Y646H


,gene,variant,drugs,clinical_significance,chromosome,start,stop,reference_bases,variant_bases,representative_transcript
0,MAP2K1,P124S,Selumetinib,Resistance,15,66729162.0,66729162.0,C,T,ENST00000307102.5
1,MAP2K1,Q56P,Selumetinib,Resistance,15,66727451.0,66727451.0,A,C,ENST00000307102.5
2,PDGFRA,D842V,Imatinib,Resistance,4,55152093.0,55152093.0,A,T,ENST00000257290.5
3,ARAF,S214C,Sorafenib,Sensitivity/Response,X,47426121.0,47426121.0,C,G,ENST00000377045.4
4,JAK2,V617F,Fedratinib,Sensitivity/Response,9,5073770.0,5073770.0,G,T,ENST00000381652.3
...,...,...,...,...,...,...,...,...,...,...
444,ABL1,G250E,Dasatinib,Sensitivity/Response,9,133738349.0,133738349.0,G,A,ENST00000318560.5
445,ABL1,H396P,Dasatinib,Sensitivity/Response,9,133750356.0,133750356.0,A,C,ENST00000318560.5
447,EGFR,G598V,Erlotinib,Sensitivity/Response,7,55233043.0,55233043.0,G,T,ENST00000275493.2
448,IDH1,R132C,Olaparib,Sensitivity/Response,2,209113113.0,209113113.0,G,A,ENST00000415913.1


In [203]:
fix_evidence_df.to_csv('../datasets/middlefile/civic_evidence_table.csv', index=None)

In [204]:
fix_evidence_df['clinical_significance'].value_counts()

Resistance              217
Sensitivity/Response    215
Reduced Sensitivity      10
Adverse Response          2
Name: clinical_significance, dtype: int64